In [ ]:
import os
import torch
import pydicom
import pandas as pd
from PIL import Image
from collections import Counter
from torchvision import transforms

# Set paths and reutilize previous transformations.
BTPDataset = r"C:\Brain-Tumor-Progression"
BTPGrades = r"C:\Users\hgood\OneDrive\Documents\Brain-Tumor-ProgressionCNNAssignments.csv"
BestModelPath = "best_model.pth"
transformREM = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
#Load ResNet model.
Device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BuildResnet = build_resnet(num_classes=3)
BuildResnet.load_state_dict(torch.load(BestModelPath, map_location=Device)['model_state_dict'])
BuildResnet.to(Device)
BuildResnet.eval()

#Function that predicts most common gades from each DICOM file in a folder.
def Folder_Grades(scan_folder):
    GradePrediction = []
    for root, _, files in os.walk(scan_folder):
        for file in files:
            if file.endswith('.dcm'):
                try:
# Convert DICOMs to RGB PIL images, add batch dimensions, get the class index and predict grades.
                    DCMBTP = pydicom.dcmread(os.path.join(root, file))
                    ConvertDCMBTP = Image.fromarray(DCMBTP.pixel_array).convert("RGB")
                    TensorImageBTP = transformREM(ConvertDCMBTP).unsqueeze(0).to(Device)
                    with torch.no_grad():
                        ArgMax = torch.argmax(BuildResnet(TensorImageBTP), dim=1).item()
                    GradePrediction.append(ArgMax)
                except Exception as e:
                    print(f"Error processing {file}: {e}")
    return Counter(GradePrediction).most_common(1)[0][0] if GradePrediction else None

#Collect predictions after iterating through the entire dataset.
FolderPredictions = []
for patientid in os.listdir(BTPDataset):
    Patient = os.path.join(BTPDataset, patientid)
    if not os.path.isdir(Patient): continue
    for folderdate in os.listdir(Patient):
        Dates = os.path.join(Patient, folderdate)
        if not os.path.isdir(Dates): continue
#Only process files if they are in T1post or FLAIR folders.
        for checkfolders in os.listdir(Dates):
            if not any(tag in checkfolders for tag in ["T1post", "FLAIR"]):
                continue
            Scans = os.path.join(Dates, checkfolders)
            FinalGrades = Folder_Grades(Scans)
            if FinalGrades is not None:
                FolderPredictions.append({
                    "PatientID": patientid,
                    "ScanDateFolder": folderdate,
                    "PredictedGrade": FinalGrades
                })
                break  
#Output predictions to a CSV file.
pd.DataFrame(FolderPredictions).to_csv(BTPGrades, index=False)
print(f"Predictions saved to {BTPGrades}")